In [ ]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
import pathlib
data_root = pathlib.Path("/root/work/mnist")
print(data_root)

/root/work/mnist


In [ ]:
def preprocess_image(image):
    image = tf.image.decode_bmp(image, channels=3)
    image = tf.image.resize(image, [28, 28])
    image /= 255.0  # normalize to [0,1] range
    return image
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)
#图像数据集
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#标签数据集
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index, name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name] for path in all_image_paths]
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
#一种图像标签数据集
image_label_ds1 = tf.data.Dataset.zip((image_ds, label_ds))
image_count = len(all_image_paths)
ds1 = image_label_ds1.shuffle(buffer_size=image_count)
ds1 = ds1.repeat()
ds1 = ds1.batch(32)
ds1 = ds1.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#另一种图像标签数据集
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label
image_label_ds2 = ds.map(load_and_preprocess_from_path_label)
ds2 = image_label_ds2.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds2 = ds2.batch(32)
ds2 = ds2.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [ ]:
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Flatten(input_shape=(28, 28, 3)))
nn.add(tf.keras.layers.Dense(128, activation='relu'))
nn.add(tf.keras.layers.Dropout(0.2))
nn.add(tf.keras.layers.Dense(10, activation='softmax'))
nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = nn.fit(
  ds1,
  #validation_data=ds2,
  epochs=10,
  steps_per_epoch=1000
)

Epoch 1/10
1000/1000 [==============================] - 62s 5ms/step - loss: 0.5155 - accuracy: 0.8439
Epoch 2/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1847 - accuracy: 0.9435
Epoch 3/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1569 - accuracy: 0.9519
Epoch 4/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1377 - accuracy: 0.9564
Epoch 5/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1180 - accuracy: 0.9652
Epoch 6/10
1000/1000 [==============================] - 6s 6ms/step - loss: 0.1030 - accuracy: 0.9684
Epoch 7/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.1027 - accuracy: 0.9683
Epoch 8/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0842 - accuracy: 0.9752
Epoch 9/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0872 - accuracy: 0.9740
Epoch 10/10
1000/1000 [==============================] - 5s 5ms/step - loss: 0.08

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=26e5d5b4-a604-4f7b-aef4-7756f8127ef0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>